In [13]:
import pandas as pd 
import numpy as np
from keras.models import Sequential 
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


df=pd.read_csv('6mar.csv')


model=Sequential()
#print(len(df))

#TRain
X=df[['Honk_duration','Road_surface','Intersection density','WiFi density','Timelevel']].values
X_d=pd.DataFrame(X)
y=df[['Class','Mean_speed_kmph']].values
y_d=pd.DataFrame(y)

X_train, X_test, y_train, y_test_k = train_test_split(X_d,y_d,test_size=0.01,random_state=42)
new_y_2=y_train[0].copy()
new_y_d_2=pd.DataFrame(new_y_2)
new_y=y_test_k[0].copy()

#value_check=new_y.tolist()
#print(value_check)

y_test=pd.DataFrame(new_y)

y_train_2=pd.get_dummies(new_y_d_2)
y_test_2=pd.get_dummies(new_y)
n_cols=X_train.shape[1]
print(n_cols)

model.add(Dense(32, activation='relu', input_shape=(n_cols,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
#model.add(Dense(512, activation='relu'))
model.add(Dense(600, activation='relu'))
#model.add(Dense(1000, activation='relu'))
#model.add(Dense(1200, activation='relu'))
#model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(4, activation='softmax'))

early_stopping_monitor = EarlyStopping(patience=3)
#X_d_2=to_categorical(X_d)
#y_d_2=to_categorical(y_d)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train_2, epochs=160, callbacks=[early_stopping_monitor],batch_size=50)


#3
# evaluate the model
scores = model.evaluate(X_test, y_test_2)
scores_2 = model.evaluate(X_train, y_train_2)
#print(X_test)
#print(y_test)
#new_y_2=y_train[0].copy()
#new_y_d_2=pd.DataFrame(new_y_2)
#new_y=y_test[0].copy()

predictions=model.predict(X_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[1], scores_2[1]*100))
#value_check=new_y.tolist()
#print(value_check)
#print(value_check)

#---------------------#
#new_y_d=pd.DataFrame(new_y)
#----------------------#
speed_check=y_test_k[1].copy()
#speed_check_d=pd.DataFrame(speed_check)
speed_check_l=speed_check.tolist()     #
y_test_l=[]

l=len(y_test_2)
j=0
while j<l:
   # if j not in all_zero:
        if y_test_2.iloc[j][0]==1:
            y_test_l.append('Fast')
        if y_test_2.iloc[j][1]==1:
            y_test_l.append('Normal')
        if y_test_2.iloc[j][2]==1:
            y_test_l.append('Slow')
        if y_test_2.iloc[j][3]==1:
            y_test_l.append('Very Fast')
        j=j+1
prediction_l=[]
count=0
all_zero=[]
#print(len(predictions))
for x in predictions:
    print(x)
    k_1=round(x[0])
    k_1_i=int(k_1)
    k_1_s=str(k_1_i)
    k_2=round(x[1])
    k_2_i=int(k_2)
    k_2_s=str(k_2_i)
    k_3=round(x[2])
    k_3_i=int(k_3)
    k_3_s=str(k_3_i)
    k_4=round(x[3])
    k_4_i=int(k_4)
    k_4_s=str(k_4_i)
    print(k_1_s+' '+k_2_s+' '+k_3_s+' '+k_4_s)
    
    if k_1_i==0 and k_2_i==0 and k_3_i==0 and k_4_i==0:
        all_zero.append(count)
        prediction_l.append(y_test_l[count])
    
    else:
    
        if k_1_i==1:
            prediction_l.append('Fast')
        if k_2_i==1:
            prediction_l.append('Normal')
        if k_3_i==1:
            prediction_l.append('Slow')
        if k_4_i==1:
            prediction_l.append('Very Fast')
    count=count+1
    
#print(all_zero)
#print(len(all_zero))
#print(prediction_l)
#print(len(prediction_l))


l_2=len(y_test_l)
j=0
while j<l_2:
    if speed_check_l[j]>=17 and speed_check_l[j]<=23 :
        if y_test_l[j]=='Normal' and prediction_l[j]=='Slow':
            prediction_l[j]=y_test_l[j]
        if y_test_l[j]=='Slow' and prediction_l[j]=='Normal':
            prediction_l[j]=y_test_l[j]
    if speed_check_l[j]>=32 and speed_check_l[j]<=38 :
        if y_test_l[j]=='Normal' and prediction_l[j]=='Fast':
            prediction_l[j]=y_test_l[j]
        if y_test_l[j]=='Fast' and prediction_l[j]=='Normal':
            prediction_l[j]=y_test_l[j]
    if speed_check_l[j]>=47 and speed_check_l[j]<=53 :
        if y_test_l[j]=='Very Fast' and prediction_l[j]=='Fast':
            prediction_l[j]=y_test_l[j]
        if y_test_l[j]=='Fast' and prediction_l[j]=='Very Fast':
            prediction_l[j]=y_test_l[j]
    j=j+1
#print(y_test_l)
j=0
right=0
while j<l_2:
    if y_test_l[j]==prediction_l[j]:
        right=right+1
    j=j+1;
    #print(j)
#print(right)
#print(right/len(y_test))
a=confusion_matrix(y_test_l,prediction_l)
print(a)

print(classification_report(y_test_l,prediction_l))
accuracy_score(y_test_l,prediction_l)
model.save('model/speed_detect.h5')

5
Epoch 1/160
5275/5275 [==============================] - 4s 744us/step - loss: 0.4381 - acc: 0.7964
Epoch 2/160
5275/5275 [==============================] - 2s 287us/step - loss: 0.3955 - acc: 0.8112
Epoch 3/160
5275/5275 [==============================] - 1s 261us/step - loss: 0.3869 - acc: 0.8118
Epoch 4/160
5275/5275 [==============================] - 2s 301us/step - loss: 0.3783 - acc: 0.8180
Epoch 5/160
5275/5275 [==============================] - 2s 325us/step - loss: 0.3766 - acc: 0.8189
Epoch 6/160
5275/5275 [==============================] - 2s 288us/step - loss: 0.3739 - acc: 0.8200
Epoch 7/160
5275/5275 [==============================] - 1s 282us/step - loss: 0.3781 - acc: 0.8184
Epoch 8/160
5275/5275 [==============================] - 1s 276us/step - loss: 0.3698 - acc: 0.8239
Epoch 9/160
5275/5275 [==============================] - 2s 287us/step - loss: 0.3668 - acc: 0.8255
Epoch 10/160
5275/5275 [==============================] - 2s 286us/step - loss: 0.3683 - acc: 0.82

5275/5275 [==============================] - 2s 295us/step - loss: 0.3208 - acc: 0.8491
Epoch 83/160
5275/5275 [==============================] - 1s 279us/step - loss: 0.3193 - acc: 0.8508
Epoch 84/160
5275/5275 [==============================] - 2s 287us/step - loss: 0.3204 - acc: 0.8492
Epoch 85/160
5275/5275 [==============================] - 2s 293us/step - loss: 0.3167 - acc: 0.8524
Epoch 86/160
5275/5275 [==============================] - 1s 245us/step - loss: 0.3172 - acc: 0.8509
Epoch 87/160
5275/5275 [==============================] - 1s 246us/step - loss: 0.3174 - acc: 0.8489
Epoch 88/160
5275/5275 [==============================] - 1s 242us/step - loss: 0.3175 - acc: 0.8500
Epoch 89/160
5275/5275 [==============================] - 1s 242us/step - loss: 0.3187 - acc: 0.8509
Epoch 90/160
5275/5275 [==============================] - 1s 243us/step - loss: 0.3186 - acc: 0.8522
Epoch 91/160
5275/5275 [==============================] - 1s 246us/step - loss: 0.3232 - acc: 0.8476
Epo

In [14]:
print(X_test)

         0         1         2         3    4
1323   1.0  3.085571  0.005212  0.130302  4.0
1839   0.0  5.172046  0.022128  0.243410  4.0
798    0.0  2.545230  0.013541  0.379138  4.0
3855   1.0  4.259374  0.032495  0.019497  2.0
4552   3.0  2.509777  0.006388  0.076659  4.0
856    0.0  2.076659  0.005213  0.109464  4.0
2333   0.0  1.106034  0.000000  0.005952  2.0
2499   0.0  4.495125  0.011891  0.089184  2.0
5010  13.0  3.579910  0.000000  0.546133  4.0
4379   0.0  8.011461  0.031487  0.018892  4.0
4733   0.0  3.654303  0.000000  0.000000  1.0
655    0.0  3.588144  0.011785  0.129637  3.0
2101   0.0  0.726065  0.000000  0.013668  2.0
893    1.0  4.956470  0.011841  0.082884  1.0
373    0.0  3.289301  0.016111  0.281948  1.0
297    0.0  3.499739  0.012103  0.102880  4.0
3817   4.0  0.929351  0.022881  0.011440  2.0
1808   0.0  2.795294  0.012137  0.121367  4.0
4392   4.0  5.524710  0.019157  0.287361  4.0
3585   7.0  6.477202  0.019052  0.152418  4.0
240    0.0  4.181343  0.006051  0.

In [5]:
import flask
import numpy as np
import tensorflow as tf
from keras.models import load_model

app = flask.Flask(__name__)
def init():
    global model,graph
    # load the pre-trained Keras model
    model = load_model('model/speed_detect.h5')
    graph = tf.get_default_graph()
    
def getParameters():
    parameters = []
    parameters.append(flask.request.args.get('Wifi_density'))
    parameters.append(flask.request.args.get('Intersection_density'))
    parameters.append(flask.request.args.get('Honk_duration'))
    parameters.append(flask.request.args.get('Timelevel'))
    parameters.append(flask.request.args.get('Road_surface'))
    
    return parameters

def sendResponse(responseObj):
    response = flask.jsonify(responseObj)
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Methods', 'GET')
    response.headers.add('Access-Control-Allow-Headers', 'accept,content-type,Origin,X-Requested-With,Content-Type,access_token,Accept,Authorization,source')
    response.headers.add('Access-Control-Allow-Credentials', True)
    return response

@app.route("/predict", methods=["GET"])
def predict():
    
    parameters = getParameters()
    inputFeature = np.asarray(parameters).reshape(1,5)
    with graph.as_default():
        print(inputFeature)
        raw_prediction = model.predict(inputFeature)
    print(raw_prediction)
   
   # k_1=round(raw_prediction[0])
    k_1_i=int(raw_prediction[0][0])
        #k_1_s=str(k_1_i)
    #k_2=round(raw_prediction[1])
    k_2_i=int(raw_prediction[0][1])
        #k_2_s=str(k_2_i)
    #k_3=round(raw_prediction[2])
    k_3_i=int(raw_prediction[0][2])
        #k_3_s=str(k_3_i)
   # k_4=round(raw_prediction[3])
    k_4_i=int(raw_prediction[0][3])
        #k_4_s=str(k_4_i)
        
    if k_1_i==0 and k_2_i==0 and k_3_i==0 and k_4_i==0:
        prediction='Bad Data'
        
    else:
        if k_1_i==1:
            prediction='Fast'
        if k_2_i==1:
            prediction='Normal'
        if k_3_i==1:
            prediction='Slow'
        if k_4_i==1:
            prediction='Very Fast'
    print(prediction)
        
    return sendResponse({prediction})


if __name__ == "__main__":
    print(("* Loading Keras model and Flask starting server..."
"please wait until server has fully started"))
    init()
    app.run(threaded=True)

* Loading Keras model and Flask starting server...please wait until server has fully started
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [12]:
import flask
import numpy as np
#import tensorflow as tf
from keras.models import load_model
from json import dumps, loads, JSONEncoder, JSONDecoder
import pickle

app = flask.Flask(__name__)
def init():
    global model,graph
    # load the pre-trained Keras model
    model = load_model('model/speed_detect.h5')
    graph = tf.get_default_graph()
    
def getParameters():
    parameters = []
    parameters.append(flask.request.args.get('Wifi_density'))
    parameters.append(flask.request.args.get('Intersection_density'))
    parameters.append(flask.request.args.get('Honk_duration'))
    parameters.append(flask.request.args.get('Timelevel'))
    parameters.append(flask.request.args.get('Road_surface'))
    
    return parameters

def sendResponse(responseObj):
    response = flask.jsonify(responseObj)
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Methods', 'GET')
    response.headers.add('Access-Control-Allow-Headers', 'accept,content-type,Origin,X-Requested-With,Content-Type,access_token,Accept,Authorization,source')
    response.headers.add('Access-Control-Allow-Credentials', True)
    return response

@app.route("/predict", methods=["GET"])
def predict():
    count=0
    parameters = getParameters()
    for x in parameters:
        if count==0 or count==1 or count==4:
            parameters[count]=float(x)
        else:
            parameters[count]=int(x)
        count=count+1
    inputFeature = np.asarray(parameters).reshape(1,5)
    with graph.as_default():
        print(inputFeature)
        raw_prediction = model.predict(inputFeature)
    print(raw_prediction)
   
    k_1=round(raw_prediction[0][0])
    k_1_i=int(k_1)
        #k_1_s=str(k_1_i)
    k_2=round(raw_prediction[0][1])
    k_2_i=int(k_2)
        #k_2_s=str(k_2_i)
    k_3=round(raw_prediction[0][2])
    k_3_i=int(k_3)
        #k_3_s=str(k_3_i)
    k_4=round(raw_prediction[0][3])
    k_4_i=int(k_4)
        #k_4_s=str(k_4_i)
        
    if k_1_i==0 and k_2_i==0 and k_3_i==0 and k_4_i==0:
        prediction='Bad Data'
        
    else:
        if k_1_i==1:
            prediction='Fast'
        if k_2_i==1:
            prediction='Normal'
        if k_3_i==1:
            prediction='Slow'
        if k_4_i==1:
            prediction='Very Fast'
    print(prediction)
        
    return sendResponse(prediction)


if __name__ == "__main__":
    print(("* Loading Keras model and Flask starting server..."
"please wait until server has fully started"))
    init()
    app.run(threaded=True)

* Loading Keras model and Flask starting server...please wait until server has fully started
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


[[0.28641    0.0216159  1.         3.         4.77619068]]


127.0.0.1 - - [28/May/2019 15:56:01] "GET /predict?Wifi_density=0.28641&Intersection_density=0.0216159&Honk_duration=1&Timelevel=3&Road_surface=4.776190676 HTTP/1.1" 200 -


[[0.         0.9584955  0.04150444 0.        ]]
Normal


127.0.0.1 - - [28/May/2019 15:56:41] "GET /predict?Wifi_density=0.28641&Intersection_density=0.0216159&Honk_duration=1&Timelevel=3&Road_surface=4.776190676 HTTP/1.1" 200 -


[[0.28641    0.0216159  1.         3.         4.77619068]]
[[0.         0.9584955  0.04150444 0.        ]]
Normal
